<a href="https://colab.research.google.com/github/Kienknu/Kienknu/blob/main/SMILES_from_chemical_space_Option_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdkit
!pip install molmass

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.7/36.7 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.8/73.8 kB 2.2 MB/s eta 0:00:00


In [2]:
import rdkit
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Draw, rdChemReactions
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
from rdkit.ML.Cluster import Butina
import re

**Comparison**

**Step 1: Loading Files**


In [9]:
import glob
import json

#------- Loading chemical_space files-----#

chemical_space = set()

file_pattern = 'simulated_chemical_space_*.txt'
chunk_files = glob.glob(file_pattern)

print(f"Found {len(chunk_files)} chunk files matching '{file_pattern}'.")

for file_path in chunk_files:
    print(f"Loading SMILES from: {file_path}")
    with open(file_path, 'r') as f:
        for line in f:
            smi = line.strip()
            if smi:
                chemical_space.add(smi)

print(f"Total unique SMILES in combined chemical_space: {len(chemical_space)}")

Found 6 chunk files matching 'simulated_chemical_space_*.txt'.
Loading SMILES from: simulated_chemical_space_chunk_3_cresol.txt
Loading SMILES from: simulated_chemical_space_chunk_5_cresol.txt
Loading SMILES from: simulated_chemical_space_chunk_0_cresol.txt
Loading SMILES from: simulated_chemical_space_chunk_4_cresol.txt
Loading SMILES from: simulated_chemical_space_chunk_2_cresol.txt
Loading SMILES from: simulated_chemical_space_chunk_1_cresol.txt
Total unique SMILES in combined chemical_space: 241005


**Step 2: Loading experimental file**

In [7]:
excel_result_file = 'Experimental_data_Geondo.xlsx'
# options for the user
options = {
    'ForestFire_1': '1',
    'ForestFire_2': '2',
    'ForestFire_3': '3',
    'normal_day_4': '4',
    'normal_day_5': '5',
    'normal_day_6': '6',
    'normal_day_7': '7',

}

# Create a sheet name for easier lookup
options_map = {value: name for name, value in options.items()}
options_str = ', '.join([f"{name}:{value}" for name, value in options.items()])

excel_sheet_name = None
while excel_sheet_name is None:
    excel_sheet_name_input = input(f"Enter number to select sheet_name ({options_str}):")

    if excel_sheet_name_input in options_map:
        excel_sheet_name = options_map[excel_sheet_name_input]
    else:
        print("Invalid number entered. Please try again.")
try:
    exp_data = pd.read_excel(excel_result_file, sheet_name=excel_sheet_name)
    print(f"Successfully loaded sheet '{excel_sheet_name}' from '{excel_result_file}'.")
except FileNotFoundError:
    print(f"Error: The file '{excel_result_file}' was not found.")
    exp_data = pd.DataFrame()
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")
    exp_data = pd.DataFrame()

while True:
    operation_mode = input('Enter the MS mode (e.g., n for Negative/ p for Positive):').upper()
    if operation_mode == "N":
        exp_data['Neu_Mass'] = exp_data['Exp m/z'] + 1.007276
        break
    elif operation_mode == "P":
        exp_data['Neu_Mass'] = exp_data['Exp m/z'] - 1.007276
        break
    else:
        print("Invalid mode. Please enter 'n' for Negative or 'p' for Positive.")

print("Original DataFrame shape:", exp_data.shape)

display(exp_data.head())

Enter number to select sheet_name (ForestFire_1:1, ForestFire_2:2, ForestFire_3:3, normal_day_4:4, normal_day_5:5, normal_day_6:6, normal_day_7:7):1
Successfully loaded sheet 'ForestFire_1' from 'Experimental_data_Geondo.xlsx'.
Enter the MS mode (e.g., n for Negative/ p for Positive):n
Original DataFrame shape: (3321, 26)


,Flag,Index,Class,DBE,Neutral DBE,Formula,Adduct,Exp m/z,Mono Abund,Total Abund,...,#C,#H,#N,#O,#S,H/C,N/C,O/C,S/C,Neu_Mass
0,+,3823,O3S,0.5,0,C4H10O3S,H,137.02756,6857.0,6857.0,...,4,10,0,3,1,2.5000,0.0,0.7500,0.2500,138.034836
1,+,3817,O3S,0.0,0,C4H10O3S,NaN,138.03541,5208.0,5208.0,...,4,10,0,3,1,2.5000,0.0,0.7500,0.2500,139.042686
2,+,3659,O2S,0.5,0,C6H14O2S,H,149.06409,3602.0,3602.0,...,6,14,0,2,1,2.3333,0.0,0.3333,0.1667,150.071366
3,+,3824,O3S,0.5,0,C5H12O3S,H,151.04342,9735.0,9735.0,...,5,12,0,3,1,2.4000,0.0,0.6000,0.2000,152.050696
4,+,3995,O3S,0.0,0,C5H12O3S,NaN,152.05127,8183.0,8183.0,...,5,12,0,3,1,2.4000,0.0,0.6000,0.2000,153.058546


**Step 3: Perform Matching Comparison**


In [10]:
all_matching_smiles_initial = set()
matching_smiles_by_excel_row_initial = {}

exp_neu_masses = exp_data['Neu_Mass'].tolist()

# Function to calculate the neutral mass
def calculate_neutral_mass_from_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return Chem.rdMolDescriptors.CalcExactMolWt(mol)
    return None

# Define a mass tolerance
mass_tolerance = 0.000002 # 2ppm

for smi in chemical_space:
    calculated_neutral_mass = calculate_neutral_mass_from_smiles(smi)

    if calculated_neutral_mass is not None:

        for j in range(len(exp_data)):
            excel_mass = exp_neu_masses[j]

            # Check for a match within the specified tolerance
            if excel_mass != 0 and abs(calculated_neutral_mass - excel_mass)/excel_mass <= mass_tolerance:

                all_matching_smiles_initial.add(smi)
                row_index = exp_data.index[j]
                if row_index not in matching_smiles_by_excel_row_initial:
                    matching_smiles_by_excel_row_initial[row_index] = []
                if smi not in matching_smiles_by_excel_row_initial[row_index]:
                    matching_smiles_by_excel_row_initial[row_index].append(smi)
                break

print(f"Size of chemical space: {len(chemical_space)}")
print(f"Number of initial matching smiles found: {len(all_matching_smiles_initial)}")

# **Filtering unrealistic products**

all_matching_smiles = set()
removed_unrealistic_count = 0

# Filtering criteria
allowed_elements = {'C', 'H', 'O', 'N','S'}

allowed_charges = {0}

for smi in all_matching_smiles_initial:
    mol = Chem.MolFromSmiles(smi)

    if mol is None:
        removed_unrealistic_count += 1
        continue

    # 1. Valence Check
    problems = Chem.DetectChemistryProblems(mol)
    if len(problems) > 0:
        removed_unrealistic_count += 1
        continue

    # 2. Allowed Atom Types Check
    unallowed_elements = {atom.GetSymbol() for atom in mol.GetAtoms()} - allowed_elements
    if unallowed_elements:
        removed_unrealistic_count += 1
        continue

    # 3. Formal Charge Check
    total_charge = Chem.GetFormalCharge(mol)
    if total_charge not in allowed_charges:
         removed_unrealistic_count += 1
         continue

    # 4. Sanitization Check
    mol_sanitized = Chem.MolFromSmiles(smi, sanitize=False)
    if mol_sanitized is not None:
        try:
            from rdkit.Chem import SanitizeFlags
            Chem.SanitizeMol(mol_sanitized, SanitizeFlags.SANITIZE_ALL)
        except Exception as e:
            removed_unrealistic_count += 1
            continue

    all_matching_smiles.add(smi)

print(f"Number of unrealistic products removed: {removed_unrealistic_count}")
print(f"Number of matching smiles after filtering: {len(all_matching_smiles)}")
print("-" * 20)

# --- Update matching_smiles_by_excel_row ---#
matching_smiles_by_excel_row = {}

for row_index, smiles_list in matching_smiles_by_excel_row_initial.items():
    filtered_smiles_for_row = [smi for smi in smiles_list if smi in all_matching_smiles]
    if filtered_smiles_for_row:
        matching_smiles_by_excel_row[row_index] = filtered_smiles_for_row

print(f"Number of Excel rows with matching filtered products: {len(matching_smiles_by_excel_row)}")
print("-" * 20)

# **Final Matching Percentage**
matching_excel_info_all = list(matching_smiles_by_excel_row.keys())

all_excel_indices = set(exp_data.index)
matching_indices_set = set(matching_excel_info_all)

matching_percentage = (len(matching_indices_set) / len(all_excel_indices)) * 100 if len(all_excel_indices) > 0 else 0
print(f"Final matching percentage: {matching_percentage:.2f}%")

Size of chemical space: 241005
Number of initial matching smiles found (based on mass): 129
Number of unrealistic products removed: 0
Number of matching smiles after filtering: 129
--------------------
Number of Excel rows with matching filtered products: 6
--------------------
Final matching percentage: 0.18%


**Saving both formulas and SMILES for the matching compounds**

In [11]:
from collections import defaultdict
from rdkit.Chem.rdMolDescriptors import CalcMolFormula

# Function to get elemental formula string from SMILES
def get_elemental_formula_string(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        formula = CalcMolFormula(mol)
        return formula
    return "Invalid_SMILES"

output_data_rows = []

for smi in all_matching_smiles:
    formula_string = get_elemental_formula_string(smi)
    if formula_string != "Invalid_SMILES":
        output_data_rows.append({
            'Elemental Formula': formula_string,
            'Matching Simulated SMILES': smi,
        })

output_df_rows = pd.DataFrame(output_data_rows)

output_df_rows = output_df_rows.sort_values(by='Elemental Formula').reset_index(drop=True)


# Name the output file
output_filename_rows = f'SMILES_for_VOC_name_vs_{excel_sheet_name}.xlsx'

# Save the file to a xlsx or csv file
try:
    output_df_rows.to_excel(output_filename_rows, index=False)
    print(f"Successfully saved file, to '{output_filename_rows}'")
except Exception as e:
    print(f"Error saving the output file: {e}")

# Display the first few rows of the new output
print("\nFirst 5 rows of the saved data:")
display(output_df_rows.head())

Successfully saved file, to 'SMILES_for_VOC_name_vs_ForestFire_1.xlsx'

First 5 rows of the saved data:


,Elemental Formula,Matching Simulated SMILES
0,C10H14O11,O=CC(O)(COCC(O)(C=O)C(O)C(=O)O)C(O)C(=O)O
1,C10H14O11,CC(C=O)(OOC(C)(C(=O)O)C(O)C=O)C(O)C(=O)OO
2,C10H14O11,CC(OC(CO)(C(=O)O)C(O)(O)C=O)(C(=O)O)C(O)C=O
3,C10H14O11,CC(OC(CO)(C(=O)O)C(O)C=O)(C(=O)O)C(O)(O)C=O
4,C10H14O11,CC(OC(COO)(C(=O)O)C(O)C=O)(C(=O)O)C(O)C=O
